In [1]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.appName("Questoes").getOrCreate()


In [2]:
df = spark.read.table('silver_db.tb_corrida_taxi_ny')
df.show()

+--------------------+-------------+------------------+----------------------+-------------+-------------------+---------------------+----------------------+-----------------------------+-----------------+------------------------+------------+--------------------------+------------------+
|                  id|id_fornecedor|   nome_fornecedor|quantidade_passageiros|valor_corrida| data_hora_embarque|data_hora_desembarque|indicador_cancelamento|indicador_viagem_sem_cobranca|id_tipo_pagamento|descricao_tipo_pagamento|tipo_servico|data_hora_criacao_registro|ano_mes_referencia|
+--------------------+-------------+------------------+----------------------+-------------+-------------------+---------------------+----------------------+-----------------------------+-----------------+------------------------+------------+--------------------------+------------------+
|b470b91a-ea8c-4e5...|            2|Curb Mobility, LLC|                     1|        16.00|2023-03-01 00:25:10|  2023-03-01 00:35

# Entendimento do dado

In [3]:
df.filter('valor_corrida < 0').show() # Existem valores negativos na coluna valor_corrida. 

+--------------------+-------------+------------------+----------------------+-------------+-------------------+---------------------+----------------------+-----------------------------+-----------------+------------------------+------------+--------------------------+------------------+
|                  id|id_fornecedor|   nome_fornecedor|quantidade_passageiros|valor_corrida| data_hora_embarque|data_hora_desembarque|indicador_cancelamento|indicador_viagem_sem_cobranca|id_tipo_pagamento|descricao_tipo_pagamento|tipo_servico|data_hora_criacao_registro|ano_mes_referencia|
+--------------------+-------------+------------------+----------------------+-------------+-------------------+---------------------+----------------------+-----------------------------+-----------------+------------------------+------------+--------------------------+------------------+
|1cf6e65e-719f-413...|            2|Curb Mobility, LLC|                     3|       -10.88|2023-05-01 00:30:22|  2023-05-01 00:36

In [11]:
_ = (
    df.filter('valor_corrida < 0')
    .groupBy('descricao_tipo_pagamento')
    .count()
    .orderBy('count')
    .show()
) # A maioria é "Contestação", portanto, os valores foram estornados para o cliente.

+------------------------+-----+
|descricao_tipo_pagamento|count|
+------------------------+-----+
|    TIPO DE PAGAMENTO...|    8|
|       Cartão de crédito|  116|
|    Viagem com tarifa...|  172|
|            Sem cobrança|25914|
|                Dinheiro|32962|
|             Contestação|83151|
+------------------------+-----+



In [9]:
# Qual a média de valor total (total_amount) recebido em um mês considerando todos os yellow táxis da frota?


_ = (
    df
    .filter("tipo_servico = 'YELLOW'") # Somente yellow táxis
    .filter("indicador_cancelamento = 'N'") # Somente corridas não canceladas
    .filter("indicador_viagem_sem_cobranca = 'N'") # Somente corridas com cobrança
    .filter("valor_corrida > 0") # Somente valores positivos
    .groupBy("ano_mes_referencia") 
    .agg(F.round(F.avg("valor_corrida"), 2).alias("media_valor_corrida"))
    .orderBy("ano_mes_referencia")
    .show()
)

+------------------+-------------------+
|ano_mes_referencia|media_valor_corrida|
+------------------+-------------------+
|           2023-01|              27.48|
|           2023-02|              27.37|
|           2023-03|              28.31|
|           2023-04|              28.80|
|           2023-05|              29.50|
+------------------+-------------------+



In [12]:
# Qual a média de passageiros (passenger_count) por cada hora do dia que pegaram táxi no mês de maio considerando todos os táxis da frota?

_ = (
    df
    .filter(F.col("ano_mes_referencia") == "2023-05")
    .withColumn("hora_embarque", F.hour(F.col("data_hora_embarque")))
    .withColumn("hora_embarque", F.col("hora_embarque").cast("string"))
    .withColumn("hora_embarque", F.lpad(F.col("hora_embarque"), 2, "0"))
    .groupBy("ano_mes_referencia", "hora_embarque")
    .agg(F.round(F.avg("quantidade_passageiros"), 2).alias("media_passageiros"))
    .orderBy("hora_embarque")
    .show()
)

+------------------+-------------+-----------------+
|ano_mes_referencia|hora_embarque|media_passageiros|
+------------------+-------------+-----------------+
|           2023-05|           00|             1.41|
|           2023-05|           01|             1.42|
|           2023-05|           02|             1.44|
|           2023-05|           03|             1.43|
|           2023-05|           04|             1.39|
|           2023-05|           05|             1.26|
|           2023-05|           06|             1.24|
|           2023-05|           07|             1.25|
|           2023-05|           08|             1.26|
|           2023-05|           09|             1.28|
|           2023-05|           10|             1.32|
|           2023-05|           11|             1.33|
|           2023-05|           12|             1.35|
|           2023-05|           13|             1.35|
|           2023-05|           14|             1.36|
|           2023-05|           15|            